### İmport sacred libraries

In [ ]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor

from sklearn import metrics
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import shapiro
from sklearn import preprocessing
from warnings import filterwarnings
filterwarnings('ignore')

from sklearn import metrics
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt


**CLIENTNUM** = Client number. Unique identifier for the customer holding the account

**Attrition_Flag** = Internal event (customer activity) variable - if the account is closed then 1 else 0

**Customer_Age** = Demographic variable - Customer's Age in Years

**Gender** = Demographic variable - M=Male, F=Female

**Dependent_count** = Demographic variable - Number of dependents

**Education_Level** = Demographic variable - Educational Qualification of the account holder (example: high school, college graduate, etc.)

**Marital_Status** = Demographic variable - Married, Single, Divorced, Unknown

**Income_Category** = Demographic variable - Annual Income Category of the account holder (< $40K, $40K - 60K, $60K - $80K, $80K-$120K, $120K >)

**Card_Category** = Product Variable - Type of Card (Blue, Silver, Gold, Platinum)

**Months_on_book** = Period of relationship with bank

**Total_Relationship_Count** = Total no. of products held by the customer

**Months_Inactive_12_mon** = No. of months inactive in the last 12 months

**Contacts_Count_12_mon** = No. of Contacts in the last 12 months

**Credit_Limit** = Credit Limit on the Credit Card

**Total_Revolving_Bal** = Total Revolving Balance on the Credit Card

**Avg_Open_To_Buy** = Open to Buy Credit Line (Average of last 12 months)

**Total_Amt_Chng_Q4_Q1** = Change in Transaction Amount (Q4 over Q1)

**Total_Trans_Amt** = Total Transaction Amount (Last 12 months)

**Total_Trans_Ct** = Total Transaction Count (Last 12 months)

**Total_Ct_Chng_Q4_Q1** = Change in Transaction Count (Q4 over Q1)

**Avg_Utilization_Ratio** = Average Card Utilization Ratio

In [ ]:
data = pd.read_csv("/kaggle/input/credit-card-customers/BankChurners.csv")
df = data[data.columns[:-2]]
df.head()

*Let's first look at the data*

In [ ]:
def Missing_Values(data):
    variable_name=[]
    total_value=[]
    total_missing_value=[]
    missing_value_rate=[]
    unique_value_list=[]
    total_unique_value=[]
    data_type=[]
    for col in data.columns:
        variable_name.append(col)
        data_type.append(data[col].dtype)
        total_value.append(data[col].shape[0])
        total_missing_value.append(data[col].isnull().sum())
        missing_value_rate.append(round(data[col].isnull().sum()/data[col].shape[0],3))
        unique_value_list.append(data[col].unique())
        total_unique_value.append(len(data[col].unique()))
    missing_data=pd.DataFrame({"Variable":variable_name,"Total_Value":total_value,\
                             "Total_Missing_Value":total_missing_value,"Missing_Value_Rate":missing_value_rate,
                             "Data_Type":data_type,"Unique_Value":unique_value_list,\
                               "Total_Unique_Value":total_unique_value})
    return missing_data.sort_values("Missing_Value_Rate",ascending=False)

In [ ]:
Missing_Values(df)

In [ ]:
df["Income_Category"].unique()

In [ ]:
df['Income_Category'].replace({'$60K - $80K': "60k_80k", 'Less than $40K': "0_40k", '$80K - $120K': "80k_120k",
                                        '$40K - $60K': "40k_60k", "$120K +": "120k_plus"}, inplace = True)
df.drop("CLIENTNUM", axis = 1, inplace = True)

In [ ]:
df["Income_Category"].unique()

### Visualization

In [ ]:
def label_function(val):
    return f'{val / 100 * len(df):.0f}\n{val:.0f}%'

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(18, 10))
df.groupby("Attrition_Flag").size().plot(kind='pie', autopct=label_function, textprops={'fontsize': 20},
                                  colors=['tomato', 'gold', 'skyblue'],ax=ax1)

df.groupby("Gender").size().plot(kind='pie', autopct=label_function, textprops={'fontsize': 20},
                                  colors=['violet', 'lime'],ax=ax2)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 15), sharey=True)
fig.suptitle('Group by Attrition Flag')
sns.countplot(x="Gender", hue = "Attrition_Flag",  data=df, ax=axes[0,0], palette="Set2")
axes[0,0].set_title("GENDER & ATTRITION FLAG")

sns.countplot(x="Income_Category", hue = "Attrition_Flag",  data=df, ax=axes[0,1], palette="Set2")
axes[0,1].set_title("INCOME CATEGORY & ATTRITION FLAG")

sns.countplot(x="Education_Level", hue = "Attrition_Flag",  data=df, ax=axes[1,0], palette="Set2")
axes[1,0].set_title("EDUCATION LEVEL & ATTRITION FLAG")

sns.countplot(x="Card_Category", hue = "Attrition_Flag",  data=df, ax=axes[1,1], palette="Set2")
axes[1,1].set_title("CARD CATEGORY & ATTRITION FLAG")


In [ ]:
Missing_Values(df)

In [ ]:
fig = plt.subplots(figsize = (5,8))
sns.boxplot(y=df["Total_Ct_Chng_Q4_Q1"])

In [ ]:
fig = plt.subplots(figsize = (5,8))
sns.boxplot(y=df["Total_Trans_Ct"])

In [ ]:
sns.kdeplot(df['Customer_Age'], shade=True)


In [ ]:
df.columns

In [ ]:
columns = ["Customer_Age", 'Dependent_count', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']
for column_name in columns:
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    df = df[~((df[column_name] < (Q1 - 3 * IQR)) |(df[column_name] > (Q3 + 3 * IQR)))]

In [ ]:
df.columns

In [ ]:
df

There were 10127 row observations before outlier cleanup. We currently have 9240 number of row

### Normality test 

In [ ]:
def shapiro_wilk(data):
    columns = df.columns
    int_columns = []
    normality_column_name = []
    normality_statistic = []
    normality_result = []
    for column_type in columns:
        if df[column_type].dtypes == "int64" or df[column_type].dtypes == "float64":
            int_columns.append(column_type)
    for column in int_columns:
        stat, p = shapiro(data[column])
        statistic = ('Statistics=%.3f, p=%.3f' % (stat, p))
        #print('Statistics=%.3f, p=%.3f' % (stat, p))
        # interpret
        alpha = 0.05
        if p > alpha:
            normality_column_name.append(column)
            normality_statistic.append(statistic)
            normality.append("Sample looks Gaussian")
                    
        else:
            normality_column_name.append(column)
            normality_statistic.append(statistic)
            normality_result.append("Sample does not look Gaussian")
        
    normality_column_name = pd.DataFrame(normality_column_name, columns =['Column Name'])
    normality_statistic = pd.DataFrame(normality_statistic, columns =['Statistic']) 
    normality_result = pd.DataFrame(normality_result, columns =['Result']) 


    result = pd.concat([normality_column_name, normality_statistic, normality_result], axis=1, sort=False)
    
    return result 

In [ ]:
shapiro_wilk(df)

In [ ]:
df_corr = df.corr()
fig, ax = plt.subplots(figsize=(10, 8))
mask = np.triu(np.ones_like(df_corr, dtype=np.bool))
# mask = mask[1:, :-1]
# corr = df_corr.iloc[1:,:-1].copy()
sns.heatmap(df_corr, mask = mask, annot = True, fmt = ".2f", cmap = sns.diverging_palette(240, 10, n=19),
           vmin = -1, vmax = 1, cbar_kws = {"shrink": .8})




There is high correlation between Avg Open To buy and Credit Limit

Credit_Limit = Credit Limit on the Credit Card

Avg_Open_To_Buy = Open to Buy Credit Line (Average of last 12 months)

In [ ]:
df[["Credit_Limit", "Avg_Open_To_Buy"]]

In [ ]:
df.drop("Credit_Limit", axis = 1, inplace = True)

# TRAIN - VALIDATION - TEST Split

prepare the data set

In [ ]:
df.dtypes

In [ ]:
df['Attrition_Flag'] = df['Attrition_Flag'].map({'Attrited Customer':1, 'Existing Customer':0})

Get dummy variables

In [ ]:
df_dummies = pd.get_dummies(df)

In [ ]:
df_dummies

In [ ]:
X = df_dummies.drop("Attrition_Flag", axis = 1)
y = df_dummies["Attrition_Flag"]

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# Train - Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Train - Validation Split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 42)

# Test = 0.20
# Validation = 0.20
# Train = 0.60

x_train_shape = X_train.shape
x_val_shape = X_val.shape
x_test_shape = X_test.shape
print("X_train shape = {}\nX_val shape = {}\nX_test shape = {}".format(x_train_shape, x_val_shape, x_test_shape))

In [ ]:
modeller = []
modelName = []

        
def dictforloop(index,accuracy,recall,f1):
    dictForPlot = []
    dictForList =[]
    
    dictForPlot.append({"Model_Name":index,"ScoreType":"accuracy","Score":accuracy})
    dictForPlot.append({"Model_Name":index,"ScoreType":"Recall_Score_0","Score":recall[0]}) 
    dictForPlot.append({"Model_Name":index,"ScoreType":"Recall_Score_1","Score":recall[1]})
    dictForPlot.append({"Model_Name":index,"ScoreType":"F1_Score_0","Score":f1[0]}) 
    dictForPlot.append({"Model_Name":index,"ScoreType":"F1_Score_1","Score":f1[1]})
    
    dictForList = {"Model_Name":index,"Accuracy":accuracy,"Recall_Score_0":recall[0],
              "Recall_Score_1":recall[1],"F1_Score_0":f1[0],"F1_Score_1":f1[1]}
    
    return dictForPlot,dictForList
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)    
def importance_plot(model_name, thresh_hold):
    Importance=pd.DataFrame({"Importance":model_name.feature_importances_*100},
                       index=X_train.columns)
    importance = Importance.sort_values(by="Importance",
                      axis=0,ascending=False).iloc[0:10,:]
    importance.sort_values(by="Importance",
                      axis=0,ascending=True).plot(kind="barh",color="r")
    plt.xlabel("Değişken Önem Düzeyleri")


    a = Importance.sort_values("Importance", ascending = False)
    display_all(a[a['Importance']>thresh_hold])

def conf_mtrx(y_test, y_pred, model): 

    cm = confusion_matrix(y_test,y_pred)    
    f, ax = plt.subplots(figsize =(5,5))
    cm = confusion_matrix(y_test,y_pred)
    sns.heatmap(cm,annot = True, linewidths=0.5, linecolor="red",fmt = ".0f",ax=ax)#,cmap=plt.cm.RdPu
    plt.xlabel("predicted y values")
    plt.ylabel("real y values")
    plt.title("\nConfusion Matrix")
    plt.show()
    
def rc_recis_scres(y_test, y_pred, algorithm_name):
    from sklearn.metrics import recall_score,precision_score,accuracy_score, f1_score

    rs=recall_score(y_test, y_pred) 
    ps=precision_score(y_test, y_pred)
    f1=f1_score(y_test, y_pred)

    print()    

def roc_auc_curve_plot(model_name, X_testt, y_testt):
    ns_probs = [0 for _ in range(len(y_testt))]
    # fit a model
    # predict probabilities
    model_probs = model_name.predict_proba(X_testt)
    # keep probabilities for the positive outcome only
    model_probs = model_probs[:, 1]
    # calculate scores
    ns_auc = roc_auc_score(y_testt, ns_probs)
    lr_auc = roc_auc_score(y_testt, model_probs)
    # summarize scores
    print('No Skill: ROC AUC=%.3f' % (ns_auc))
    print( ': ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_testt, ns_probs)
    model_fpr, model_tpr, _ = roc_curve(y_testt, model_probs)
    # plot the roc curve for the model
    pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    pyplot.plot(model_fpr, model_tpr, marker='.', label='Logistic')
    # axis labels
    pyplot.xlabel('False Positive Rate')
    pyplot.ylabel('True Positive Rate')
    # show the legend
    pyplot.legend()
    # show the plot
    pyplot.show() 
    
def plot_roc_curve(fpr, tpr):
    plt.figure(figsize=(5,5))
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr,tpr, label = roc_auc_score) 
    plt.plot([0,1],ls='--')
    plt.plot([0,0],[1,0],c='.5')
    plt.plot([1,1],c='.5')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()   
    
def ML_Algorithms(model,alg_name, x_trainn,x_testt, y_trainn, y_testt):
    from sklearn.metrics import roc_auc_score,roc_curve
    
    modell = model()
    result_model = modell.fit(x_trainn, y_trainn)
    y_pred = result_model.predict(x_testt)    
    conf_mtrx(y_testt, y_pred, model)
    print("*****",alg_name," ALGORITHM:") 
    rc_recis_scres(y_testt, y_pred, alg_name)
    
    
    
    modeller.append(result_model)
    modelName.append(alg_name)
    
    print("classification_report\n",classification_report(y_testt, y_pred))
    print("Accuracy Score for ", alg_name ,accuracy_score(y_testt, y_pred))
    
    tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
    y_probs = modell.predict_proba(x_testt)[:,1] # This will give you positive class prediction probabilities  
    y_pred = np.where(y_probs > 0.5, 1, 0) # This will threshold the probabilities to give class predictions.

    fpr, tpr, thresholds = roc_curve(y_test, y_probs)

    probs = modell.predict_proba(X_test)
    probs = probs[:, 1]
    
    print()
    auc = roc_auc_score(y_test, probs)
    print('AUC: %.2f' % auc)
    #plot_roc_curve(fpr, tpr)
    return model  

In [ ]:
ML_Algorithms(RandomForestClassifier, "Random Forest Classifier", X_train, X_val, y_train, y_val)

In [ ]:
ML_Algorithms(DecisionTreeClassifier, "Decision Tree Classifier", X_train, X_val, y_train, y_val)

In [ ]:
ML_Algorithms(GradientBoostingClassifier, "Gradient Boosting Classifier", X_train, X_val, y_train, y_val)

In [ ]:
ML_Algorithms(XGBClassifier, "XGB Classifier", X_train, X_val, y_train, y_val)

In [ ]:
ML_Algorithms(LGBMClassifier, "LGBM Classifier", X_train, X_val, y_train, y_val)

In [ ]:
ML_Algorithms(CatBoostClassifier, "CatBoost Classifier", X_train, X_val, y_train, y_val)

In [ ]:
dictForPlot=[]
dictForDf = []
for i in range(len(modeller)):
    isim = modelName[i]
    y_pred = modeller[i].predict(X_val)
    
    ac = accuracy_score(y_val, y_pred)
    recallscore = recall_score(y_val, y_pred,average=None)
    f1score = f1_score(y_val, y_pred,average=None)
    
    szFp,szFl=dictforloop(isim,ac,recallscore,f1score)
    dictForPlot=dictForPlot + szFp
    dictForDf.append(szFl)
    
dictPlotDf = pd.DataFrame(dictForPlot)
dictListDf = pd.DataFrame(dictForDf)

figg=plt.figure(figsize=(20,7))
plt.title('Accuracy Rates Of Models');    

sns.barplot(x="ScoreType",y = "Score",hue ="Model_Name" , data = dictPlotDf);
dictListDf.head()

##### I will focus on 2 models for  tuned phase and these are Random Forest and LGBM

# Model Tuning 

# Random Forest

In [ ]:
rf = RandomForestClassifier()
rf_model = rf.fit(X_train, y_train)
y_pred = rf_model.predict(X_val)



print("classification_report\n",classification_report(y_val, y_pred))
print("Accuracy Score for ",accuracy_score(y_val, y_pred))

In [ ]:
rf = RandomForestClassifier(class_weight ="balanced")
rf_model = rf.fit(X_train, y_train)
y_pred = rf_model.predict(X_val)



print("classification_report\n",classification_report(y_val, y_pred))
print("Accuracy Score for ",accuracy_score(y_val, y_pred))

In [ ]:
importance_plot(rf_model, 1)


In [ ]:
n_estimators = [100, 300, 500, 800, 1200, 2000]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10, 15] 




grid_param = dict(n_estimators = n_estimators, max_depth = max_depth, 
                  min_samples_split = min_samples_split, 
                  min_samples_leaf = min_samples_leaf,
                  )

rf_tuned = GridSearchCV(rf, grid_param, cv = 5, verbose = 2, 
                      n_jobs = -1)
rf_tuned = rf_tuned.fit(X_train, y_train)

In [ ]:
rf_tuned.best_params_

rf_tuned.best_params_ = (max_depth= 15, min_samples_leaf= 1, min_samples_split= 10, n_estimators= 800)

In [ ]:
rf = RandomForestClassifier(max_depth= 15, min_samples_leaf= 1, min_samples_split= 10, n_estimators= 800)
rf_model = rf.fit(X_train, y_train)
y_pred = rf_model.predict(X_val)



print("classification_report\n",classification_report(y_val, y_pred))
print("Accuracy Score for ",accuracy_score(y_val, y_pred))

In [ ]:
tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
y_probs = rf_model.predict_proba(X_val)[:,1] # This will give you positive class prediction probabilities  
y_pred = np.where(y_probs > 0.35, 1, 0) # This will threshold the probabilities to give class predictions.

print("classification_report\n",classification_report(y_val, y_pred))
print("Accuracy Score for ",accuracy_score(y_val, y_pred))

## Time to get test with test data

In [ ]:
final_model_name = []
final_model_ac = []
final_model_recall = []
final_model_f1 = []

ac = accuracy_score(y_test, y_pred)
recallscore = recall_score(y_test, y_pred,average=None)
f1score = f1_score(y_test, y_pred,average=None)

In [ ]:
rf = RandomForestClassifier(max_depth= 15, min_samples_leaf= 1, min_samples_split= 10, n_estimators= 800)
rf_model = rf.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
y_probs = rf_model.predict_proba(X_test)[:,1] # This will give you positive class prediction probabilities  
y_pred = np.where(y_probs > 0.35, 1, 0) # This will threshold the probabilities to give class predictions.


ac_rf = accuracy_score(y_test, y_pred)
recallscore_rf = recall_score(y_test, y_pred,average=None)
f1score_rf = f1_score(y_test, y_pred,average=None)

final_model_name.append("Random Forest")
final_model_ac.append(ac_rf)
final_model_recall.append(recallscore_rf)
final_model_f1.append(f1score_rf)


print("classification_report\n",classification_report(y_test, y_pred))
print("Accuracy Score for ",accuracy_score(y_test, y_pred))



##### Random Forest
Accuracy Score for 0.9502164502164502,

Recall Score is 0.89 for 1

In [ ]:
roc_auc_curve_plot(rf_model,X_test, y_test)

#### Feature İmportance

In [ ]:
importance_plot(rf_model, 1)

# LGBM

In [ ]:
lgbm = LGBMClassifier()
lgbm_model = lgbm.fit(X_train, y_train)
y_pred = lgbm_model.predict(X_val)

print("classification_report\n",classification_report(y_val, y_pred))
print("Accuracy Score for ",accuracy_score(y_val, y_pred)),

In [ ]:
importance_plot(lgbm_model, 1000)

In [ ]:
lgbm_params = {'learning_rate': np.arange(0.0, 1.0, 0.1),
                 'max_depth': [5,10,15,25,30,40,50,70,100],
                 'num_leaves': [10,20,30,40,50,60,70,90,100,120],
                 'feature_fraction': np.arange(0.0, 1.0, 0.1),
                 'subsample': np.arange(0.2, 1.0, 0.1)}



lgbm_tuned = GridSearchCV(lgbm, lgbm_params, cv = 5, verbose = 2, 
                      n_jobs = -1)
lgbm_tuned = lgbm_tuned.fit(X_train, y_train)

In [ ]:
lgbm_tuned.best_params_

 lgbm_tuned.best_params_ = {'feature_fraction': 0.7,
 'learning_rate': 0.5,
 'max_depth': 15,
 'num_leaves': 20,
 'subsample': 0.2}

In [ ]:
lgbm = LGBMClassifier(feature_fraction= 0.7, learning_rate= 0.5, max_depth= 15, num_leaves= 20, subsample= 0.2)
lgbm_model = lgbm.fit(X_train, y_train)
y_pred = lgbm_model.predict(X_val)


print("classification_report\n",classification_report(y_val, y_pred))
print("Accuracy Score for ",accuracy_score(y_val, y_pred))

In [ ]:
tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
y_probs = lgbm_model.predict_proba(X_val)[:,1] # This will give you positive class prediction probabilities  
y_pred = np.where(y_probs > 0.5, 1, 0) # This will threshold the probabilities to give class predictions.

print("classification_report\n",classification_report(y_val, y_pred))
print("Accuracy Score for ",accuracy_score(y_val, y_pred))

In [ ]:
lgbm = LGBMClassifier(feature_fraction= 0.7, learning_rate= 0.5, max_depth= 15, num_leaves= 20, subsample= 0.2)
lgbm_model = lgbm.fit(X_train, y_train)
y_pred = lgbm_model.predict(X_val)

tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
y_probs = lgbm_model.predict_proba(X_val)[:,1] # This will give you positive class prediction probabilities  
y_pred = np.where(y_probs > 0.40, 1, 0) # This will threshold the probabilities to give class predictions.


print("classification_report\n",classification_report(y_val, y_pred))
print("Accuracy Score for ",accuracy_score(y_val, y_pred))

### Time to get test with test data

In [ ]:
lgbm = LGBMClassifier(feature_fraction= 0.7, learning_rate= 0.5, max_depth= 15, num_leaves= 20, subsample= 0.2)
lgbm_model = lgbm.fit(X_train, y_train)
y_pred = lgbm_model.predict(X_test)

tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
y_probs = lgbm_model.predict_proba(X_test)[:,1] # This will give you positive class prediction probabilities  
y_pred = np.where(y_probs > 0.40, 1, 0) # This will threshold the probabilities to give class predictions.


ac_lgbm = accuracy_score(y_test, y_pred)
recallscore_lgbm = recall_score(y_test, y_pred,average=None)
f1score_lgbm = f1_score(y_test, y_pred,average=None)

final_model_name.append("LGBM")
final_model_ac.append(ac_lgbm)
final_model_recall.append(recallscore_lgbm)
final_model_f1.append(f1score_lgbm)


print("classification_report\n",classification_report(y_test, y_pred))
print("Accuracy Score for ",accuracy_score(y_test, y_pred))

##### LGBM
Accuracy Score for 0.9702380952380952,

Recall Score is 0.90 for 1

In [ ]:
roc_auc_curve_plot(lgbm_model,X_test, y_test)

In [ ]:
final_model_name_df = pd.DataFrame(final_model_name, columns=['Model Name'])
final_model_ac_df = pd.DataFrame(final_model_ac, columns=['Model Accuracy Score'])
final_model_recall_df = pd.DataFrame(final_model_recall, columns=['Recall  0', "Recall 1"])
final_model_f1_df = pd.DataFrame(final_model_f1, columns=['F1  0', "F1 1"])
final_model_result = pd.concat([final_model_name_df, final_model_ac_df, final_model_recall_df, final_model_f1_df], axis = 1)
final_model_result

As a result our best model is LGBM and Model Accuracy Score 0.97023 ~ **97**, For 1's Recall is 0.897516 ~ **90**.